In [11]:
import os
from dotenv import load_dotenv
import mysql.connector

In [12]:
# Load .env variables
load_dotenv()

# Get DB credentials from environment variables
db_config = {
    "host": os.getenv("MYSQL_HOST"),
    "port": int(os.getenv("MYSQL_PORT", 3306)),  # default 3306 if not set
    "user": os.getenv("MYSQL_USER"),
    "password": os.getenv("MYSQL_PASSWORD"),
    "database": os.getenv("MYSQL_DATABASE")
}

# Query

In [22]:
user_query = "show all tables"

In [38]:
database_schema = f"""
-- ==========================
-- 1. Table Definitions
-- ==========================

-- Doctor Table
CREATE TABLE Doctor (
    AadharID CHAR(12) PRIMARY KEY,
    Name VARCHAR(100),
    Specialty VARCHAR(100),
    YearsOfExperience INT
);

-- Patient Table
CREATE TABLE Patient (
    AadharID CHAR(12) PRIMARY KEY,
    Name VARCHAR(100),
    Address VARCHAR(200),
    BirthDate DATE,
    PrimaryPhysicianID CHAR(12),
    FOREIGN KEY (PrimaryPhysicianID) REFERENCES Doctor(AadharID)
);

-- PharmaceuticalCompany Table
CREATE TABLE PharmaceuticalCompany (
    Name VARCHAR(100) PRIMARY KEY,
    PhoneNo VARCHAR(15)
);

-- Drug Table
CREATE TABLE Drug (
    TradeName VARCHAR(100),
    Formula TEXT,
    CompanyName VARCHAR(100),
    PRIMARY KEY (TradeName, CompanyName),
    FOREIGN KEY (CompanyName) REFERENCES PharmaceuticalCompany(Name) ON DELETE CASCADE
);

-- Pharmacy Table
CREATE TABLE Pharmacy (
    Name VARCHAR(100) PRIMARY KEY,
    Address VARCHAR(200),
    PhoneNo VARCHAR(15)
);

-- Sells Table
CREATE TABLE Sells (
    PharmacyName VARCHAR(100),
    TradeName VARCHAR(100),
    CompanyName VARCHAR(100),
    Price DECIMAL(10,2),
    PRIMARY KEY (PharmacyName, TradeName, CompanyName),
    FOREIGN KEY (PharmacyName) REFERENCES Pharmacy(Name),
    FOREIGN KEY (TradeName, CompanyName) REFERENCES Drug(TradeName, CompanyName)
);

-- Prescription Table
CREATE TABLE Prescription (
    DoctorID CHAR(12),
    PatientID CHAR(12),
    Date DATE,
    PRIMARY KEY (DoctorID, PatientID, Date),
    FOREIGN KEY (DoctorID) REFERENCES Doctor(AadharID),
    FOREIGN KEY (PatientID) REFERENCES Patient(AadharID)
);

-- PrescriptionDrug Table
CREATE TABLE PrescriptionDrug (
    DoctorID CHAR(12),
    PatientID CHAR(12),
    Date DATE,
    TradeName VARCHAR(100),
    CompanyName VARCHAR(100),
    Quantity INT,
    PRIMARY KEY (DoctorID, PatientID, Date, TradeName, CompanyName),
    FOREIGN KEY (DoctorID, PatientID, Date) REFERENCES Prescription(DoctorID, PatientID, Date),
    FOREIGN KEY (TradeName, CompanyName) REFERENCES Drug(TradeName, CompanyName)
);

-- Contract Table
CREATE TABLE Contract (
    PharmaCompanyName VARCHAR(100),
    PharmacyName VARCHAR(100),
    StartDate DATE,
    EndDate DATE,
    Content TEXT,
    SupervisorAadharID CHAR(12),
    PRIMARY KEY (PharmaCompanyName, PharmacyName, StartDate),
    FOREIGN KEY (PharmaCompanyName) REFERENCES PharmaceuticalCompany(Name),
    FOREIGN KEY (PharmacyName) REFERENCES Pharmacy(Name),
    FOREIGN KEY (SupervisorAadharID) REFERENCES Doctor(AadharID)
);


"""

In [39]:
import ollama

prompt = f"""
You are an expert SQL query generator. Given a natural language request about data from the database, write a valid MySQL SQL query that answers the request.

Instructions:  
- Only generate SELECT, SHOW and USE queries unless otherwise specified.  
- Use proper SQL syntax for MySQL.  
- Make sure to handle filtering, sorting, and aggregation as per the request.  
- If the request is ambiguous, ask for clarification or make a reasonable assumption.  
- Don't include explanations, only output the SQL query.

Example input:  
"Show me the names and emails of users who signed up after January 1, 2023."

Expected output:  
SELECT name, email FROM users WHERE signup_date > '2023-01-01';

Database scheme is also provided to make a better analysis of the user's needs.
Database schema: {database_schema}

The user's query: {user_query}

"""

response = ollama.chat(
    model='qwen3:0.6b',
    messages=[{'role': 'user', 'content': prompt}]
)

print(response['message']['content'])

<think>
Okay, the user wants to "show all tables" from the database. Let me check the database schema provided.

Looking at the tables, there are several tables listed: Doctor, Patient, PharmaceuticalCompany, Drug, Pharmacy, Sells, Prescription, PrescriptionDrug, and Contract. All of them are defined in the table definitions. 

The user's request is straightforward: they want to list all tables. In MySQL, to show all tables, you can use the SHOW TABLES statement. That's the standard way to get a list of tables in a database. 

I don't need to filter, sort, or aggregate anything here. The example given in the instructions doesn't require any of those, so just the SHOW TABLES query is needed. 

I should make sure there are no typos in the table names. The tables listed are all correct. So the SQL query should be SHOW TABLES FROM ... but since the user didn't specify a database, maybe it's implied, but the question says "database", so perhaps the default is the current one. However, the u

In [25]:
## cleaning 
import re

raw_content = response['message']['content']

# Remove all <think>...</think> tags and their content
clean_content = re.sub(r'<think>.*?</think>', '', raw_content, flags=re.DOTALL)

clean_content = clean_content.strip()

generated_query = clean_content
print(generated_query)

SHOW TABLES;


In [34]:
# Connect to MySQL
try:
    conn = mysql.connector.connect(**db_config)
    print("Connected to MySQL database!\n")

    cursor = conn.cursor()

    # Example query
    cursor.execute(generated_query)
    results = cursor.fetchall()
    for row in results:
        print(row)

    cursor.close()
    conn.close()

except mysql.connector.Error as err:
    print("Error:", err)


Connected to MySQL database!

('Amoxicillin', 'C16H19N3O5S', 'PharmaY')
('Drug1', 'F1', 'PharmaX')
('Drug2', 'F2', 'PharmaX')
('Drug3', 'F3', 'PharmaX')
('Drug4', 'F4', 'PharmaX')
('Drug5', 'F5', 'PharmaX')
('Drug6', 'F6', 'PharmaX')
('Drug7', 'F7', 'PharmaX')
('Ibuprofen', 'C13H18O2', 'PharmaX')
('Paracetamol', 'C8H9NO2', 'PharmaX')


In [37]:
print(results)

[('Amoxicillin', 'C16H19N3O5S', 'PharmaY'), ('Drug1', 'F1', 'PharmaX'), ('Drug2', 'F2', 'PharmaX'), ('Drug3', 'F3', 'PharmaX'), ('Drug4', 'F4', 'PharmaX'), ('Drug5', 'F5', 'PharmaX'), ('Drug6', 'F6', 'PharmaX'), ('Drug7', 'F7', 'PharmaX'), ('Ibuprofen', 'C13H18O2', 'PharmaX'), ('Paracetamol', 'C8H9NO2', 'PharmaX')]
